In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/Sun_lab')

Mounted at /content/drive


In [ ]:
#change to your data path
data_path = '/content/drive/My Drive/Sun_lab/extra/Spring2024/stock_price/completed_trials/'


In [ ]:
#replace with your file name. Columns should be 'name' and 'ticker'.
ticker_name_file = '541_tickers_from_top_1000.csv'

In [ ]:
import pandas as pd
ticker_dict = pd.read_csv(data_path+ticker_name_file, index_col=0)
len(ticker_dict)

541

In [ ]:
#get stock_prices_historical.csv by running tickers_2_history.ipynb first

all_data = pd.read_csv(data_path + 'stock_prices_historical.csv', index_col = 0)

In [ ]:
all_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,name
0,1980-03-28 00:00:00-05:00,0.000000,0.148410,0.138056,0.138056,2400.0,0.0,0.0,GSK,GlaxoSmithKline
1,1980-03-31 00:00:00-05:00,0.138056,0.138056,0.138056,0.138056,0.0,0.0,0.0,GSK,GlaxoSmithKline
2,1980-04-01 00:00:00-05:00,0.138056,0.138056,0.138056,0.138056,0.0,0.0,0.0,GSK,GlaxoSmithKline
3,1980-04-02 00:00:00-05:00,0.000000,0.148410,0.138056,0.138056,800.0,0.0,0.0,GSK,GlaxoSmithKline
4,1980-04-03 00:00:00-05:00,0.138056,0.138056,0.138056,0.138056,0.0,0.0,0.0,GSK,GlaxoSmithKline


In [ ]:
#read sponsors data
sponsors = pd.read_csv(data_path + 'sponsors_completed_all_phases.csv')

#read study data
studies = pd.read_csv(data_path + 'studies_completed_all_phases.csv')

In [ ]:
#merge
sponsors_studies = pd.merge(sponsors, studies, on='nct_id')

sponsors_studies.shape

In [ ]:
sponsors_studies = sponsors_studies[sponsors_studies['name'].isin(all_data.name.tolist())]

In [ ]:
sponsors_studies['Ticker'] = sponsors_studies['name'].map(ticker_dict)

In [ ]:
all_data['Date'] = pd.to_datetime(all_data['Date'], errors='coerce')

# Extract only the date part for all rows in the 'Date' column
all_data['Date'] = all_data['Date'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None)

# Display the DataFrame with the modified 'Date' column
all_data

In [ ]:
def calculate_sma_slope(df, ticker, given_date, window_size, days):
    """
    Calculate the slope of Simple Moving Average (SMA) within a specified number of days from a given date for a given ticker.

    Parameters:
    - df: DataFrame containing historical stock prices with 'Date', 'Ticker', and 'Close' columns.
    - ticker: Ticker symbol of the stock.
    - given_date: Date for which to calculate the slope of SMA within the specified window.
    - window_size: Number of days for the SMA window.
    - days: number of days after the trial completion date

    Returns:
    - slope: Slope of SMA within the specified window from the given date for the given ticker.
    """
    # Convert given_date to a Timestamp object if it's in string format
    if isinstance(given_date, str):
        given_date = pd.to_datetime(given_date)

    # Filter DataFrame for the given ticker
    df_ticker = df[df['Ticker'] == ticker]


    if not isinstance(df_ticker['Date'].iloc[0], pd.Timestamp):
        df_ticker['Date'] = pd.to_datetime(df_ticker['Date'])

    df_ticker = df_ticker.sort_values(by='Date')

    selected_dates = df_ticker['Close'][df_ticker['Date'] > given_date]

    # Calculate SMA using rolling method only for dates after the given date
    sma = selected_dates.rolling(window=window_size, min_periods = 1).mean()


    # Calculate the slope of SMA within the specified window after the given date
    slope = (sma.iloc[days-1] - sma.iloc[0]) / window_size

    return slope


In [ ]:
#apply on trials to calculate slope
sponsors_studies['completion_date'] = pd.to_datetime(sponsors_studies['completion_date'])
sponsors_studies['Slope'] = sponsors_studies.apply(lambda row: calculate_sma_slope(all_data[all_data['Ticker'] == row['Ticker']], row['Ticker'], row['completion_date'], 5, 7), axis=1)


In [ ]:
# Group slopes by their sign (positive or negative)
slope_groups = sponsors_studies.groupby(sponsors_studies['Slope'] > 0)

# Calculate statistics
slope_stats = slope_groups['Slope'].agg(['count', 'mean', 'std', 'min', 'max'])

# Plot distribution of positive and negative slopes
plt.figure(figsize=(8, 6))
slope_groups['Slope'].count().plot(kind='bar', color=['red' if group else 'blue' for group in slope_groups.groups.keys()])
plt.xlabel('Slope Sign')
plt.ylabel('Count')
plt.title('Distribution of Positive and Negative Slopes')
plt.xticks(ticks=[0, 1], labels=['Negative', 'Positive'], rotation=0)
plt.show()

# Print statistics
print("Statistics of Positive and Negative Slopes:")
print(slope_stats)

In [ ]:
sponsors_studies.to_csv(data_path+'slope_labels.csv')